# 深度可分离卷积性能分析

aim：看看在得到相同维度的output的情况下，使用**常规卷积的框架**和**深度卷积+逐点卷积（这个在mobilenet的论文里其实叫depthwise conv和pointwise conv）**

深度可分离卷积框架就用mobilenetv1(因为只看了一眼v1TAT)，常规卷积就根据mobilenetv1的尺寸进行设计。

In [71]:
#mobilenetv1模型构建

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.autograd import Variable

## 构建mobileNet_v1
mobileNet_v1用深度可分离卷积代替了常规卷积，实现了参数量和计算量的大幅减少

In [53]:
#depthwiseconv构建
class depthwiseSeprsbleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(depthwiseSeprsbleConv, self).__init__()

        #设计模型模块
        self.depthwiseConv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)  #深度卷积部分的input和output的通道数是相同的
        self.pointwiseConv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.BN1 = nn.BatchNorm2d(in_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.BN2 = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwiseConv(x)
        x = self.BN1(x)
        x = self.relu(x)
        x = self.pointwiseConv(x)
        #x = self.depthwiseConv(x)
        x = self.BN2(x)
        x = self.relu(x)
        return x

In [54]:
#mobilenetv1构建
class mobileNetv1(nn.Module):
    def __init__(self, num_classes):
        super(mobileNetv1, self).__init__()

        #设计模型模块

        #常规卷积模块
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.BN1 = nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)

        self.dw_conv1 = depthwiseSeprsbleConv(32, 64)
        self.dw_conv2 = depthwiseSeprsbleConv(64, 128)
        self.dw_conv3 = depthwiseSeprsbleConv(128, 128)
        self.dw_conv4 = depthwiseSeprsbleConv(128, 256)
        self.dw_conv5 = depthwiseSeprsbleConv(256, 256)
        self.dw_conv6 = depthwiseSeprsbleConv(256, 512)
        self.dw_conv7 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv8 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv9 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv10 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv11 = depthwiseSeprsbleConv(512, 512)
        self.dw_conv12 = depthwiseSeprsbleConv(512, 1024)
        self.dw_conv13 = depthwiseSeprsbleConv(1024, 1024)

        self.Avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.BN1(x)
        x = self.relu(x)

        x = self.dw_conv1(x)
        x = self.dw_conv2(x)
        x = self.dw_conv3(x)
        x = self.dw_conv4(x)
        x = self.dw_conv5(x)
        x = self.dw_conv6(x)
        x = self.dw_conv7(x)
        x = self.dw_conv8(x)
        x = self.dw_conv9(x)
        x = self.dw_conv10(x)
        x = self.dw_conv11(x)
        x = self.dw_conv12(x)
        x = self.dw_conv13(x)

        x = self.Avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

        

## 构建相同维度的常规卷积backbone
用于和mobileNet_v1进行比较，实现和mobileNet_v1相同的维度变化，需要s1和s2

In [59]:
#normal_conv_s1构建
class normal_conv_s1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(normal_conv_s1, self).__init__()

        #设计模型模块
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride=1, padding=1)
        self.BN = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.relu(x)
        return x

In [60]:
#normal_conv_s2构建
class normal_conv_s2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(normal_conv_s2, self).__init__()

        #设计模型模块
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride=2, padding=1)
        self.BN = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.relu(x)
        return x

In [64]:
#同一size的常规卷积backbone
class Normal_conv(nn.Module):
    def __init__(self, num_classes):
        super(Normal_conv, self).__init__()

        ##backbone
        
        self.conv1 = normal_conv_s2(3, 32)
        self.conv2 = normal_conv_s1(32, 64)
        self.conv3 = normal_conv_s2(64, 128)
        self.conv4 = normal_conv_s1(128, 128)
        self.conv5 = normal_conv_s2(128, 256)
        self.conv6 = normal_conv_s1(256, 256)
        self.conv7 = normal_conv_s2(256, 512)
        self.conv8 = normal_conv_s1(512, 512)
        self.conv9 = normal_conv_s1(512, 512)
        self.conv10 = normal_conv_s1(512, 512)
        self.conv11 = normal_conv_s1(512, 512)
        self.conv12 = normal_conv_s1(512, 512)
        self.conv13 = normal_conv_s2(512, 1024)

        self.Avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.Avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

## 载入数据集
这里我们先用CIFAR10数据集试一试，CIFAR10数据集的resolution是32×32×3，经过backbone之后，resolution会降为原来的1/32，所以理论上最后的Avg_pool层没啥用，反正先跑一下试试吧，谁知道呢，224*224的数据集后面再看看，imagenet太大了，咱train不起啊

In [55]:
#定义模型参数
CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if CUDA else "cpu")
#print(DEVICE)

mobileNet_v1 = mobileNetv1(10).to(DEVICE)
#print(mobileNet_v1)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mobileNet_v1.parameters(), lr=0.001)

In [48]:
for name, p in mobileNet_v1.named_parameters():
    if 'conv1' in name:
        print(name, ' :', p.shape)



conv1.weight  : torch.Size([32, 3, 3, 3])
conv1.bias  : torch.Size([32])
dw_conv1.depthwiseConv.weight  : torch.Size([32, 1, 3, 3])
dw_conv1.depthwiseConv.bias  : torch.Size([32])
dw_conv1.pointwiseConv.weight  : torch.Size([64, 32, 1, 1])
dw_conv1.pointwiseConv.bias  : torch.Size([64])
dw_conv1.BN1.weight  : torch.Size([32])
dw_conv1.BN1.bias  : torch.Size([32])
dw_conv1.BN2.weight  : torch.Size([64])
dw_conv1.BN2.bias  : torch.Size([64])
dw_conv10.depthwiseConv.weight  : torch.Size([512, 1, 3, 3])
dw_conv10.depthwiseConv.bias  : torch.Size([512])
dw_conv10.pointwiseConv.weight  : torch.Size([512, 512, 1, 1])
dw_conv10.pointwiseConv.bias  : torch.Size([512])
dw_conv10.BN1.weight  : torch.Size([512])
dw_conv10.BN1.bias  : torch.Size([512])
dw_conv10.BN2.weight  : torch.Size([512])
dw_conv10.BN2.bias  : torch.Size([512])
dw_conv11.depthwiseConv.weight  : torch.Size([512, 1, 3, 3])
dw_conv11.depthwiseConv.bias  : torch.Size([512])
dw_conv11.pointwiseConv.weight  : torch.Size([512, 512, 1

In [68]:
#train_data = CIFAR10('cifar', train=True, download=True,  transform=transforms.ToTensor())  经典SSL报错
#载入CIFAR10数据集
import torchvision

train_data = torchvision.datasets.CIFAR10(
    root = "E:/dataset/",   #根据数据集实际位置修改地址
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)
data = DataLoader(train_data, batch_size=1, shuffle=True)
#Dataloader后的数据能不能可视化一下呢？

Files already downloaded and verified


### mobileNet_v1 train

In [72]:
import time

for epoch in range(5):
    start = time.time()
    for img,label in data:
        print(img.shape[0])
        img = Variable(img).to(DEVICE)
        label = Variable(label).to(DEVICE)
        output = mobileNet_v1(img)
        loss = criterion(output, label)

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

        pre = torch.argmax(output,1)
        print(pre)
        num = (pre == label).sum().item()
        acc = num / img.shape[0]
        print(acc)
    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("epoch:",epoch + 1)
    print("loss:",loss.item())
    print("acc:",acc)
pass

1


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 3.43 GiB already allocated; 0 bytes free; 3.45 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### normal_conv train

In [62]:

normalconv = Normal_conv(10).to(DEVICE)
optimizer = torch.optim.Adam(normalconv.parameters(), lr=0.001)

In [69]:
for epoch in range(5):
    normalconv.train()
    start = time.time()
    for img,label in data:
        print(img.shape[0])
        img = Variable(img).to(DEVICE)
        label = Variable(label).to(DEVICE)
        output = normalconv(img)
        loss = criterion(output, label)

        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

        pre = torch.argmax(output,1)
        print(pre)
        num = (pre == label).sum().item()
        acc = num / img.shape[0]
        print(acc)

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("epoch:",epoch + 1)
    print("loss:",loss.item())
    print("acc:",acc)
pass

1


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 1024, 1, 1])